In [1]:
from PyPDF2 import PdfReader
import chromadb
PATH = "pdf/JavaBasico.pdf"


reader = PdfReader(PATH)
        
text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")

In [2]:
text_split = []
metadata=[]
chunk_size = 512
chunk_overlap = 100
        
for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})
    

In [3]:
import ollama

for i, d in enumerate(text_split):
        response = ollama.embeddings(model="mxbai-embed-large",
                                       prompt=d)
        
        collection.add(
                ids=[str(i)],
                embeddings=response['embedding'],
                documents=[d],
                metadatas=metadata[i]
    )

In [5]:
chat_history = []

for i in range(2):

    prompt = input("Pergunte algo")

    response = ollama.embeddings(
        prompt=prompt,
        model="mxbai-embed-large"
    )

    results = collection.query(
        query_embeddings=[response["embedding"]],
        n_results=2,
        where={"name": "book-langchain"}
    )
    
    data = results['documents'][0]
    distance = results['distances']

    print(distance)
    

    for k in range(2):
        if distance[0][k] > 120:
            data = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
    print("\n")
    print(data)
    output = ollama.generate(
        model="llama2",
        prompt=f"""You are a chatbot assistant. 

                Answer in portuguese the question based only on the context below:    
                {data}

        """
    )
    output = output['response']
    print(output)
    chat_history.append(f"Human: {prompt}\nAi: {output}")

    response = ollama.embeddings(model="mxbai-embed-large",
                                       prompt=chat_history[i])

    history_collection.add(
                ids=[str(i)],
                embeddings=response['embedding'],
                documents=[chat_history]
    )
    print(history_collection)

Pergunte algo o que é classe em orientação a objeto


[[106.46578216552734, 107.82245635986328]]


['NTARES SOBRE CLASSES E \nORIENTAÇÃO A OBJETO\nVamos explicar mais alguns conceitos complementares sobre \nclasses e orientação a objeto aplicando-os a classes que implementam programação gráﬁ  ca.\nO primeiro conceito que vamos ver é o de anonymous inner \nclass , que é uma forma de simpliﬁ  carmos uma classe já escrita por \nmeio da criação de classes internas sem as mesmas estarem declaradas; funciona como se fosse uma chamada de método em que você coloca sua implementação em outra parte por questão de organização. Vamos ver a mesma classe anterior, mas agora com uma implementação de anonymous inner class :192 Governança: Gestão, Auditoria e Tecnologia da InformaçãoJava Básico e Orientação a Objeto  | Progr', 'são acessíveis tanto pela própria classe quanto por qualquer classe que tente acessá-los.\n\x81  Quando utilizado o modiﬁ  cador de acesso protected , os ele-\nmentos da classe são acessíveis somente por classes no mesmo pacote da 

Add of existing embedding ID: 0
Insert of existing embedding ID: 0


Pergunte algo responda em portugues o que é classe em orientação a objeto


[[118.79826354980469, 120.49310302734375]]


Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta.
 Claro! Então, como assistente de chatbot, posso dizer que não tenho acesso a informações sobre a pergunta que você me fez. Desculpe, não posso fornecer mais detalhes neste momento. Is there anything else I can help you with?


Add of existing embedding ID: 1
Insert of existing embedding ID: 1


 Desculpe, como assistente de chatbot, tenho acesso limitado às informações e não posso fornecer respostas detalhadas sobre uma pergunta específica. Peço desculpas pela falta de informação.
